In [1]:
import sys
import os
from source import plotFunctions
from timeit import default_timer as timer
import numpy as np
from experiments import setup
from source import metrics
from experiments.methods import svm
from experiments.methods import proposed_gmm_core_extraction
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import Audio, display



class Experiment():
    def __init__(self, method):
        #commom for all experiments
        self.method = method
        self.initialLabeledDataPerc=0.05
        self.usePCA=False
        #used only by gmm and cluster-label process
        self.densityFunction='gmm'
        self.excludingPercentage = 0.7
        self.K = 5
        self.classifier='cluster_and_label'
        #used in alpha-shape version only
        self.CP=0.65
        self.alpha=0.5
        #used in kmeans_svm and compose only
        self.useSVM=False
        self.isImbalanced=False


def doExperiments(dataValues, dataLabels, experiments, numberOfTimes, batches):
    sizeOfBatch = 200#int(len(dataLabels)/batches)
       
    for name, e in experiments.items():
        CoreX = []
        CoreY = []
        elapsedTime = []
        accTotal = []
        accuracies=[]
        e.classes=list(set(dataLabels)) 
        e.sizeOfBatch = sizeOfBatch
        e.batches = batches
        e.dataLabels = dataLabels
        e.dataValues = dataValues
        e.clfName = 'svm'#cl, svm, rf
        
        for i in range(numberOfTimes):
            start = timer()
            #accuracy per step
            accuracies, CoreX, CoreY = e.method.start(dataValues=e.dataValues, dataLabels=e.dataLabels, usePCA=e.usePCA, classes=e.classes, classifier=e.classifier, densityFunction=e.densityFunction, batches=e.batches, sizeOfBatch = e.sizeOfBatch, initialLabeledDataPerc=e.initialLabeledDataPerc, excludingPercentage=e.excludingPercentage, K=e.K, CP=e.CP, alpha=e.alpha, clfName=e.clfName)
            end = timer()
            averageAccuracy = np.mean(accuracies)
            
            #elapsed time per step
            elapsedTime.append(end - start)
            
            accTotal.append(averageAccuracy)
        #print("Total of ", numberOfTimes, " experiment iterations with an average accuracy of ", np.mean(accTotal))
        
        #print data distribution in step t
        initial = (batches*sizeOfBatch)-sizeOfBatch
        final = initial + sizeOfBatch
        plotFunctions.plot(dataValues[initial:final], dataLabels[initial:final], CoreX, CoreY, batches)
        print("\n\n")
        print("{} batches of {} instances".format(batches, sizeOfBatch))
        print("Average execution time: ", np.mean(elapsedTime))
        plotFunctions.finalEvaluation(accuracies, batches)
        

def main():
    experiments = {}
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'experiments/data'+sep
    
    #loading a dataset
    #dataValues, dataLabels = setup.loadNOAADataset(path)
    dataValues, dataLabels, nameDataset = setup.load2CDT(path, sep)
        
    '''
    SVM Linear Kernel
    '''
    experiments[1] = Experiment(svm)
    
    
    ''' Proposed Method 1 (GMM extracting core supports) '''
    experiments[2] = Experiment(proposed_gmm_core_extraction)
    
    
    #running pywidget
    def run(batches):
        doExperiments(dataValues, dataLabels, experiments, 1, batches)
        
    v = interact(run, batches=(1, 80, 1));
    display(v)

    
if __name__ == "__main__":
    main()

<function __main__.main.<locals>.run>